In [27]:
# Some imports you will need
import os
import mlflow
import jwt
import json
import warnings
import sys

import pandas as pd
import numpy as np
import randomname
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
# os.environ['MLFLOW_TRACKING_URI'] = 'http://mlflow.mlv2flow2431-platform:5000'
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])
print('MLFLOW_TRACKING_TOKEN: ' + os.environ['MLFLOW_TRACKING_TOKEN'])

client = mlflow.tracking.MlflowClient()

myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

# experiment_name = date_time_str+'-'+'DEMO'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
distinct = randomname.get_name()
experiment_name = date_time_str + '-' + distinct
model_name = distinct +'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name

print(experiment_name)

experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name, tags={"mlflow.note.content": randomname.get_name(), "mlflow.domino.project_id": "1234"})
    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
experiment_id=experiment.experiment_id    
mlflow.set_experiment(experiment_name=experiment_name)


MLFLOW_TRACKING_URI: http://mlflow.domino-platform:5000
MLFLOW_TRACKING_TOKEN: eyJhbGciOiJIUzI1NiJ9.CnsKImRvbWlub19hcGlfa2V5IjogIjY1Zjk2Yzk4YjExMzE4MGZmNmI0MzMyOGNmMTVhMzdiM2JmODI1YTlkZTFlYjNkOTFhOGQ0MjZhNWJkYTllZTgiLAoiZG9taW5vX3J1bl9pZCI6ICI2MzkzOGQ3MjhmZjBhZDYxZjVlN2VhNzIiLAoiZG9taW5vX3Byb2plY3RfaWQiOiAiNjM5MmJjZmY5ZTNhNTEzMzUzMmFlYWU0IiwKImRvbWlub19wcm9qZWN0X25hbWUiOiAicXVpY2stc3RhcnQiLAoidGFncyI6IHsKICAiRE9NSU5PX1BST0pFQ1RfT1dORVIiOiAiaW50ZWdyYXRpb24tdGVzdCIsCiAgIm1sZmxvdy5zb3VyY2UudHlwZSI6ICJOT1RFQk9PSyIKfX0K.Ufi10giYIpJkZH8yDaqN-9GDMRldgDZ-Wa9w-f1licU
12-15-2022-mature-texture
Creating experiment 
12-15-2022-mature-texture


<Experiment: artifact_location='http://mlflow.domino-platform:5000/api/2.0/mlflow-artifacts/artifacts/mlflow', creation_time=1671064556981, experiment_id='26', last_update_time=1671064556981, lifecycle_stage='active', name='12-15-2022-mature-texture', tags={'domino.project': 'quick-start',
 'mlflow.domino.environment_id': '63913cf66d2650476805c785',
 'mlflow.domino.environment_revision_id': '63913cf66d2650476805c787',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project': 'quick-start',
 'mlflow.domino.project_id': '6392bcff9e3a5133532aeae4',
 'mlflow.domino.project_name': 'quick-start',
 'mlflow.domino.run_number': '1',
 'mlflow.domino.user': 'integration-test',
 'mlflow.note.content': 'internal-elk',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'integration-test'}>

In [28]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
# csv_url = "/mnt/winequality-red.csv"

try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.3
l1_ratio = 0.6
while(alpha<1):
    with mlflow.start_run() as parent_run:
        print('--- Start Run ---')
        print("run_id: {}".format(parent_run.info.run_id))
        print("description: {}".format(parent_run.data.tags.get("mlflow.note.content")))
        print("version tag value: {}".format(parent_run.data.tags.get("version")))
        print('Alpha : ' + str(alpha))
        print('L1_Ratio : ' + str(l1_ratio))
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("foo", alpha * l1_ratio)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print("mlflow tracking uri:", mlflow.get_tracking_uri())
        print("mlflow tracking type:", tracking_url_type_store)
        
        # mlflow.log_artifact("/tmp/test.txt")
        # mlflow.log_artifact("/mnt/rush")
        
        mlflow.log_text("Randomly generated file", randomname.get_name())
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(mlflow.get_tracking_uri())
        print(tracking_url_type_store)
        # Model registry does not work with file store
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
        mlflow.sklearn.log_model(lr, "model", registered_model_name=model_name)
        
        with mlflow.start_run(nested=True) as child_run:
            mlflow.log_param("user", "ethan")            
            mlflow.log_metric("alpha", alpha)
            mlflow.log_param("supercalifragilisticexpialidociousparam", "supercalifragilisticexpialidociousvalue")
            mlflow.log_metric("supercalifragilisticexpialidociousmetric", 0.99999999999)
            with mlflow.start_run(nested=True) as grandchild_run:
                mlflow.log_param("grandchild", "true")            
                mlflow.log_metric("supercalifragilisticexpialidociousmetric", 0.99999999999)
            

        with mlflow.start_run(nested=True) as child_run:
            mlflow.log_param("user", "noah")
            mlflow.log_metric("r2", r2)
            mlflow.log_param("supercalifragilisticexpialidociousparam", "supercalifragilisticexpialidociousvalue")
            mlflow.log_metric("supercalifragilisticexpialidociousmetric", 0.888888888888888888)            
            
        alpha=alpha+0.1
        l1_ratio = l1_ratio + 0.05
        
        
        # Search all (grand)child runs with a parent id by using special tag mlflow.domino.root_run_id
        query = "tags.mlflow.domino.root_run_id = '{}'".format(parent_run.info.run_id)
        results = mlflow.search_runs(experiment_ids=[experiment_id], filter_string=query)
        print("(grand)child runs:")
        print(results[["run_id", "tags.mlflow.runName"]])
        
        
        
        print("--- End Run ---")

--- Start Run ---
run_id: aa64554a267a4ffd8519f795ed0e46d8
description: None
version tag value: None
Alpha : 0.3
L1_Ratio : 0.6
Elasticnet model (alpha=0.300000, l1_ratio=0.600000):
  RMSE: 0.7383627912277975
  MAE: 0.5879367459403766
  R2: 0.17358681741383186
mlflow tracking uri: http://mlflow.domino-platform:5000
mlflow tracking type: http
http://mlflow.domino-platform:5000
http


Successfully registered model 'mature-texture-quick-start'.
2022/12/15 00:36:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mature-texture-quick-start, version 1
Created version '1' of model 'mature-texture-quick-start'.


(grand)child runs:
                             run_id tags.mlflow.runName
0  98a8adcdbf024e1e98bc1c5be2f77506   serious-skink-240
1  13ac0bd169674ecc8b2d536ae15bfd19  colorful-whale-460
2  69c4eb55d68742c995c2cd02313db0ea   silent-dolphin-32
--- End Run ---
--- Start Run ---
run_id: 534759da54e846109e75d4463998e093
description: None
version tag value: None
Alpha : 0.4
L1_Ratio : 0.65
Elasticnet model (alpha=0.400000, l1_ratio=0.650000):
  RMSE: 0.7608445462306436
  MAE: 0.6146195888110327
  R2: 0.12249521307589506
mlflow tracking uri: http://mlflow.domino-platform:5000
mlflow tracking type: http
http://mlflow.domino-platform:5000
http


Registered model 'mature-texture-quick-start' already exists. Creating a new version of this model...
2022/12/15 00:36:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mature-texture-quick-start, version 2
Created version '2' of model 'mature-texture-quick-start'.


(grand)child runs:
                             run_id  tags.mlflow.runName
0  91116feb3fa24071a93c9e071f754e31         sassy-ox-537
1  99b552d81f304f058816a4bf9362db41   likeable-snail-732
2  3c9cf9246dd64da9a54fd39525a1f319  likeable-shrike-777
--- End Run ---
--- Start Run ---
run_id: 7b8d4b7591354229a220292544ae3ef5
description: None
version tag value: None
Alpha : 0.5
L1_Ratio : 0.7000000000000001
Elasticnet model (alpha=0.500000, l1_ratio=0.700000):
  RMSE: 0.7928645248345049
  MAE: 0.649004845336213
  R2: 0.04708183950348932
mlflow tracking uri: http://mlflow.domino-platform:5000
mlflow tracking type: http
http://mlflow.domino-platform:5000
http


...

(grand)child runs:
                             run_id   tags.mlflow.runName
0  2e751a80945b46c088a7b9f422bf2d96  selective-shrimp-908
1  821592498ec34d1ca01e2e3cd14a0e16       sneaky-calf-964
2  0a4f57710a3c402a9efe0a4b44761a35       unique-grub-717
--- End Run ---
--- Start Run ---
run_id: c1e7555e83fa4a1b807ca8c15a1b1927
description: None
version tag value: None
Alpha : 0.8999999999999999
L1_Ratio : 0.9000000000000002
Elasticnet model (alpha=0.900000, l1_ratio=0.900000):
  RMSE: 0.8025392279052322
  MAE: 0.6622068020806747
  R2: 0.023684531910865947
mlflow tracking uri: http://mlflow.domino-platform:5000
mlflow tracking type: http
http://mlflow.domino-platform:5000
http


Registered model 'mature-texture-quick-start' already exists. Creating a new version of this model...
2022/12/15 00:36:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mature-texture-quick-start, version 7
Created version '7' of model 'mature-texture-quick-start'.


(grand)child runs:
                             run_id tags.mlflow.runName
0  d8750bfde1ba4b36828d631eefd86027   lyrical-panda-687
1  d3b1ca2cfdac4dc0bb52f89ad50c4401   unleashed-ape-907
2  a6d6ce3241c34dfe8171e63027abdd14  mercurial-toad-895
--- End Run ---
--- Start Run ---
run_id: 82633debe21c49d183d6ddd73cf401d1
description: None
version tag value: None
Alpha : 0.9999999999999999
L1_Ratio : 0.9500000000000003
Elasticnet model (alpha=1.000000, l1_ratio=0.950000):
  RMSE: 0.8024778865005812
  MAE: 0.6632095667141477
  R2: 0.023833773895175492
mlflow tracking uri: http://mlflow.domino-platform:5000
mlflow tracking type: http
http://mlflow.domino-platform:5000
http


Registered model 'mature-texture-quick-start' already exists. Creating a new version of this model...
2022/12/15 00:36:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: mature-texture-quick-start, version 8
Created version '8' of model 'mature-texture-quick-start'.


(grand)child runs:
                             run_id   tags.mlflow.runName
0  614a94b466cd4085952f7b4dd8960090    honorable-boar-475
1  af5b733d682647b1ba7e3130aef8b21d  welcoming-shrimp-548
2  a9ccd395ede24e5cbb46394f5ed2de31     likeable-gnat-916
--- End Run ---


In [29]:
import requests

def print_artifact_info(artifact):
    print('----')
    print("artifact: {}".format(artifact.path))
    print("is_dir: {}".format(artifact.is_dir))
    print("size: {}".format(artifact.file_size))
    
e = mlflow.get_experiment_by_name(experiment_name)
print(e.artifact_location)
results = mlflow.search_runs(experiment_ids=[e.experiment_id])

# print(results)
print(e)
for index, run in results.iterrows():
    run_id = run["run_id"]
    print(f"Run ID: {run_id}")
    print(f'Artifact URI: {run["artifact_uri"]}')
    # print(run)
    ls = client.list_artifacts(run_id)
    for l in ls:
        print_artifact_info(l)
#     uri = run["artifact_uri"] + "/test.txt"
#     print(f"Downloading {uri}")
#     # response = requests.get(uri, headers={'Authorization': 'Bearer ' + os.environ['MLFLOW_TRACKING_TOKEN']})
#     # print(f"Response: {response}")

http://mlflow.domino-platform:5000/api/2.0/mlflow-artifacts/artifacts/mlflow
<Experiment: artifact_location='http://mlflow.domino-platform:5000/api/2.0/mlflow-artifacts/artifacts/mlflow', creation_time=1671064556981, experiment_id='26', last_update_time=1671064556981, lifecycle_stage='active', name='12-15-2022-mature-texture', tags={'domino.project': 'quick-start',
 'mlflow.domino.environment_id': '63913cf66d2650476805c785',
 'mlflow.domino.environment_revision_id': '63913cf66d2650476805c787',
 'mlflow.domino.hardware_tier': 'small-k8s',
 'mlflow.domino.project': 'quick-start',
 'mlflow.domino.project_id': '6392bcff9e3a5133532aeae4',
 'mlflow.domino.project_name': 'quick-start',
 'mlflow.domino.run_number': '1',
 'mlflow.domino.user': 'integration-test',
 'mlflow.note.content': 'internal-elk',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'integration-test'}>
Run ID: 614a94b466cd4085952f7b4dd8960090
Artifact URI: http://mlflow.domino-platform:5000/api/2.0/mlflow-artifacts/artifact